In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.models import load_model                                                            
import sys
import timeit
import random
import argparse
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle   
from CUI098_custom_layers import Time2Vec, PeriodicEmbed, GradAccumModel, GlobalAvgPooling1DMask
from CUI098_custom_layers import GlobalMaxPooling1DMask,PositionalEmbedding, BilinearMHAttention, PositionalEmbedding
from CUI020_seq_loader import EmbeddingSequence
import CUI099_utils as utils
from tensorflow.keras.utils import Sequence  # to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import quantile_transform

In [ ]:
cwd = os.getcwd()
proj_dir = os.path.abspath(os.path.join(cwd, os.pardir))
sys.path.append(proj_dir)
sys.path.insert(0, os.path.join(cwd, os.pardir))

In [ ]:
import rtdl
sys.modules['rtdl.rtdl'] = rtdl
from rtdl.rtdl.data import piecewise_linear_encoding,compute_piecewise_linear_encoding, PiecewiseLinearEncoder

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
os.environ["TF_CUDNN_DETERMINISTIC"]="1"
physical_devices = tf.config.list_physical_devices('GPU')
for gpu_dev in physical_devices:
    tf.config.experimental.set_memory_growth(gpu_dev, True)
physical_devices

In [ ]:
def read_data(data_path):
    if data_path.split('.')[-1] in ['txt', 'csv']:
        dat = pd.read_csv(data_path)
    elif data_path.split('.')[-1] == 'pickle':
        with open(data_path, 'rb') as f:
            dat = pickle.load(f)
    elif data_path.split('.')[-1] == 'pkl':
        with open(data_path, 'rb') as f:
            dat = pickle.load(f)
    return dat

In [ ]:
study_id = 'encounter_id'
time_since = 'hours_since_admit'
outcome = 'first_stage2_48hrs'
model_type = 'ts'

In [ ]:
#Embedding mechanism

class EmbeddingSequence(Sequence):
    """Generates data for Keras
    A child class from Sequence requires two methods: __len__ and __getitem__
    The method __getitem__ should return a complete batch
    """
    def __init__(self, x_in, y_in, list_IDs, max_ts_len=None, max_cui_len=None, batch_size=1, shuffle=True):
        """Initialization
        :param x_in: pandas dataframe with each row containing the predictor observations at a single timepoint 
            - timeseries data AND/OR processed CUIs
        :param y_in: pandas dataframe with each row being an outcome at a specific timepoint
        :param list_IDs: list of all 'label' ids to use in the generator (typically list of encounter/study ids)
        :param max_ts_len: Max number of timesteps a batch can be padded to 
        :param max_cui_len: Max number of CUIs a batch can be padded to 
        :param batch_size: batch size at each iteration
        :param shuffle: True to shuffle label indexes after every epoch
        """
        self.x_in = x_in
        self.y_in = y_in
        self.batch_size = batch_size
        self.list_IDs = list_IDs
        self.shuffle = shuffle
        self.indexes = np.arange(len(self.list_IDs))
        self.on_epoch_end()
        if max_ts_len is None:
            self.max_ts_len = 0
        else:
            self.max_ts_len = max_ts_len
        if max_cui_len is None:
            self.max_cui_len = 0
        else:
            self.max_cui_len = max_cui_len
        self.time_dim = 't2v'
        self.outcome = outcome
        self.study_id = study_id
        self.time_var = time_since

    def __len__(self):
        """Denote the number of batches per epoch
        :return: number of batches per epoch
        """
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def on_epoch_end(self):
        """Updates indexes after each epoch
        on_epoch_end is triggered once at the very beginning as well as at the end of each epoch
        """
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle:
            np.random.shuffle(self.indexes)
            
    def __data_generation(self, list_IDs_temp):
        # Generate data
        batch_X = self.x_in.loc[self.x_in[self.study_id].isin(list_IDs_temp)]
        batch_y = self.y_in.loc[self.y_in[self.study_id].isin(list_IDs_temp)]

        batch_X['timestep'] = batch_X.groupby(self.study_id).cumcount()
        time_cols = [self.time_var]
        
        X_output = []
        if model_type in ['ts', 'combo']:
            # ts input
            batch_ts = batch_X.drop([*time_cols, 'cui'], axis=1, errors='ignore')
            batch_ts = (batch_ts.set_index([self.study_id, 'timestep']).groupby(level=0).apply(
                lambda x: list(x.values)[-self.max_ts_len:])).tolist()
            batch_ts = pad_sequences(batch_ts, value = -1, padding='post', dtype='float32')
            batch_ts = batch_ts.astype('float32')
            X_output.append(batch_ts)
        
        batch_X_time = batch_X.set_index([self.study_id, 'timestep']
                                             ).groupby(level=0)[self.time_var].apply(
        lambda x: list(x)[-self.max_ts_len:])
        batch_X_time = pad_sequences(batch_X_time, value = -1, padding='post', dtype='float32')
        X_output.append(batch_X_time)

        batch_y['timestep'] = batch_y.groupby(self.study_id).cumcount()
        batch_y = batch_y.set_index([self.study_id, 'timestep']).groupby(level=0)[self.outcome].apply(
            lambda x: list(x)[-self.max_ts_len:])
        
        batch_y = pad_sequences(batch_y, padding='post', value=-1, dtype='float32')
        batch_y = np.expand_dims(batch_y, axis=-1)
        batch_y = batch_y.astype('float32')
        

        return X_output, batch_y

    def __getitem__(self, index):
        """Generate one batch of data
        :param index: index of the batch
        :returns X and y
            X is a list of X inputs to the model containing a combination of 
            structured, CUI, and timestep data to be use as input to the model
        """
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]
        
        X,y = self.__data_generation(list_IDs_temp)
        
        return X, y

### Read in model and data

In [ ]:
data_dir = '../../Data/Analytic/aki_data_20240411-1019'
model_dir = './'
model_path = 'copy20240628-0910_best_model.h5'
c_obs_path = 'copy20240621-1215_model_objects.pickle'

In [ ]:
#Load model objects
transformer = read_data('copy20240610-1226_data_transformer.pickle')
custom_obs = read_data(os.path.join(model_dir, c_obs_path))
model = load_model(os.path.join(model_dir, model_path), 
                      custom_objects=custom_obs)

In [ ]:
combo_dict = read_data('new_input2.pkl')
combo_dat = combo_dict['wrapper_df']['combined_dat']
all_test_encs = combo_dict['wrapper_df']['test_encs']
all_test_dat = combo_dat.loc[combo_dat[study_id].isin(all_test_encs)]

In [ ]:
test_dat = all_test_dat
all_cols = combo_dict['orig_dat_noise'].columns
median_val = np.expand_dims(np.median(test_dat[:100], axis=0), axis=0)

In [ ]:
# create input with median values as baseline
med_baseline_data = pd.DataFrame(median_val, columns=combo_dict['wrapper_df']['combined_dat'].columns)
med_baseline_data[study_id] = 101
med_baseline_data[time_since] = 0

In [ ]:
pred_vars = combo_dict['wrapper_df']['pred_vars']

In [ ]:
combo_dict_background = read_data('real_background.pkl')
dat_background = combo_dict_background['wrapper_df']['combined_dat']

In [ ]:
# create list of transformed columns from features
col_sort = dat_background[combo_dict['wrapper_df']['pred_vars']].nunique()
col_sort = pd.DataFrame(col_sort, columns =['count'])
col_sort.loc[col_sort['count']>10, 'count']=10
col_sort_dict = col_sort.to_dict()['count']
col_list = []
col_index_list = []
for col in dat_background[combo_dict['wrapper_df']['pred_vars']].columns:
    col_list.extend([col] * col_sort_dict[col])
    col_index_list.extend(list(range(col_sort_dict[col])))
    
col_list2 = []
for i in range(len(col_list)):
    col_list2.append(f"{col_list[i]}_{col_index_list[i]}")

In [ ]:
# testing all data
test_dat = all_test_dat
test_encs = all_test_encs

#defining some default values
outcome = 'outcome_ward24hr'
cuis = [0]

In [ ]:
def integrated_gradients(model, baseline, original, y, num_steps=50):
    
    # create inputs that incrementally change from baseline to original input
    interpolated_inputs = []
    for i in range(len(original)):
        interpolated_inputs.append(tf.linspace(baseline[i], original[i], num_steps+1)[1:])
    
    # run each new input through the model to obtain gradients
    gradients = []
    for s in range(num_steps):
        trans_dat = pd.DataFrame(transformer.transform(np.array(interpolated_inputs[0][s])))
        trans_dat[study_id] =  list(y_test[study_id])[0]
        trans_dat[time_since] = interpolated_inputs[1][s]
        test_generator = EmbeddingSequence(trans_dat, y, [list(y_test[study_id])[0]], max_cui_len=2000, 
                                           max_ts_len=2000, batch_size=1, shuffle=False)
        inputs, extra = test_generator.__getitem__(0)
        n_time = original[0].shape[0]
        inputs_tf = [tf.convert_to_tensor(inputs[0]), tf.ones(shape=(1,n_time,1)),tf.convert_to_tensor(inputs[1])]
        with tf.GradientTape() as tape:
            #fix to have variable number of inputs
            tape.watch(inputs_tf)
            output = model(inputs_tf)
            
        grad = tape.gradient(output, inputs_tf)
        gradients.append(grad)
    gradients = [[y for y in x if y is not None] for x in gradients]
    
    avg_gradients = [tf.reduce_mean(g, axis=0) for g in zip(*gradients)]
    
    return avg_gradients

In [ ]:
med_baseline_data2 = pd.DataFrame(
    np.repeat(med_baseline_data, 200, axis=0).astype(np.float32), 
    columns=all_cols
)

In [ ]:
time_start=timeit.default_timer()

num_steps = 5000
int_grad_dfs = []
feature_int_grads = []
int_grads_ts = []

for n in range(len(test_encs)):
    test_n = test_dat.loc[test_dat[study_id] == test_encs[n]]
    y_test = test_n[[study_id, time_since, outcome]]
    X_test_n = tf.convert_to_tensor(test_n[pred_vars])
    test_n_time = tf.convert_to_tensor(test_n[time_since], dtype=float)
    
    baseline = tf.convert_to_tensor(med_baseline_data2[pred_vars].iloc[:X_test_n.shape[0], :])
    baseline_time = tf.convert_to_tensor(0, dtype=float)
    
    avg_gradients = integrated_gradients(model, 
                                         [baseline, baseline_time, cuis], 
                                         [X_test_n,test_n_time,cuis], 
                                         y_test, 
                                         num_steps=num_steps)
    
    # get differences between original and baseline after transformation
    orig_transformed = transformer.transform(np.array(X_test_n))
    base_transformed = transformer.transform(np.array(baseline))
    trans_diffs = [orig_transformed-base_transformed]
    # time baseline is 0, so the difference is equal to the original values
    trans_diffs.append(test_n_time)
    
    int_grads = {}
    for i in range(len(trans_diffs)):
        int_grads[i] = tf.convert_to_tensor([trans_diffs[i]], 'float32') * avg_gradients[i]
        
    int_grad_df = pd.DataFrame(int_grads[0][0].numpy(), columns = col_list2)
    int_grad_dfs.append(int_grad_df)
    feature_igs = int_grad_df.T.reset_index(names='feature')
    feature_igs['feature'] = feature_igs['feature'].apply(lambda x: str(x)[0:-2])
    feature_igs = feature_igs.groupby('feature').sum().T.reset_index(drop=True)
    
    feature_int_grads.append(feature_igs)
    int_grad_ts = pd.DataFrame(int_grads[1][0].numpy(), columns =['time_grad'])
    int_grads_ts.append(int_grad_ts)

#show time taken
print(np.round((timeit.default_timer()-time_start)/60, 2)) 

In [ ]:
all_feat_grads = pd.concat(feature_int_grads, axis=0)
all_feat_grads[pred_vars].abs().mean().sort_values(ascending=False)[:20]

In [ ]:
#generate the explanations
IG_importances = []

for case_num, enc_id in enumerate(combo_dict['wrapper_df']['combined_dat']['encounter_id'].unique()):
    print(f"case_num: {case_num}")
    original_data = combo_dict['wrapper_df']['combined_dat'][combo_dict['wrapper_df']['combined_dat']['encounter_id'] == enc_id]
    print(f"original_data: {original_data.shape}")
    
    importance_holder = pd.DataFrame(np.zeros_like(original_data), columns=combo_dict['wrapper_df']['combined_dat'].columns)
    importance_vals = feature_int_grads[case_num]
    
    for col_v in importance_vals.columns:
        importance_holder[col_v] = importance_vals[col_v]
        
    IG_importances.append({
        'case': case_num,
        'original': original_data,
        'importance': importance_holder,
    }) 
        

In [ ]:
#save the explanations
pickle.dump(IG_importances, open('trial22_vis_outs/IntegratedGradients', 'wb'))